In [37]:
#!pip install llama_index
#!pip install chardet
#!pip install --upgrade llama-index
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [12]:
from llama_index.readers.file import PandasCSVReader
import nest_asyncio
nest_asyncio.apply()
import chardet

In [21]:
import pandas as pd

# Detect file encoding
#with open("data/movies.csv", "rb") as f:
    #raw_data = f.read()
    #result = chardet.detect(raw_data)
    #detected_encoding = result["encoding"]

#print(f"Detected Encoding: {detected_encoding}")

# Load CSV file
#csv_reader = PandasCSVReader()
#csv_reader = PandasCSVReader()  # Use alternate encoding
df = pd.read_csv("data/movies.csv")


In [22]:
#Convert CSV data to Nodes
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import Document

# Convert each row into a Document

documents = []
for _, row in df.iterrows():
    # Join row into a single string or format nicely
    content = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    metadata = {"title": row.get("original_title"), "genre": row.get("genres")}
    documents.append(Document(text=content))

# Parse each document into a node (1:1 mapping in this case)
parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(documents)


In [23]:
from llama_index.core import VectorStoreIndex, StorageContext
storage_dir = "./movie-index-latest"
#print(nodes)
index = VectorStoreIndex(nodes)
#storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
index.storage_context.persist(persist_dir=storage_dir)  # Store index for later use


In [6]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o")

In [2]:
from llama_index.core import StorageContext, load_index_from_storage
#loading movie index
storage_dir = "./movie-index-latest"
storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
# Load the index
index = load_index_from_storage(storage_context)

In [7]:
print(index.as_query_engine().query("Recommend me some sci-fi movies with a rating above 8.0."))

Some sci-fi movies with a rating above 8.0 that you might enjoy are "Blade Runner," "Inception," and "Interstellar."


In [8]:
mindex = index.as_query_engine()

In [119]:
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow

from llama_index.core.workflow import Context
from llama_index.core import StorageContext, load_index_from_storage

# 1. Search movie details
async def search_movie_details(ctx: Context, movie_name: str) -> str:
    query_result = mindex.query(f"Find details for the movie {movie_name}")
    result_str = query_result.response if hasattr(query_result, "response") else str(query_result)

    if "not found" in result_str.lower():
        return f"❌ Movie details not found for {movie_name}."

    current_state = await ctx.get("state")
    current_state["movie_details"][movie_name] = result_str
    await ctx.set("state", current_state)

    return f"✅ Movie details found for {movie_name}.\n{result_str}"


async def get_movie_budget_and_revenue(ctx: Context, movie_name: str) -> str:
    current_state = await ctx.get("state")

    # If details missing, hand off to SearchAgent
    if movie_name not in current_state["movie_details"]:
        await ctx.request_handoff(
            to_agent="SearchAgent",
            reason=f"Movie details missing for '{movie_name}', required before fetching budget/revenue."
        )
        return f"🔄 Handoff triggered to SearchAgent for '{movie_name}'"

    # Query both
    budget = mindex.query(f"What is the budget of {movie_name}?")
    revenue = mindex.query(f"What is the revenue of {movie_name}?")

    budget_str = budget.response if hasattr(budget, "response") else str(budget)
    revenue_str = revenue.response if hasattr(revenue, "response") else str(revenue)

    # Update state
    current_state["budget_details"][movie_name] = budget_str
    current_state["revenue_details"][movie_name] = revenue_str
    await ctx.set("state", current_state)

    return (
        f"💰 Budget for '{movie_name}': {budget_str}\n"
        f"💸 Revenue for '{movie_name}': {revenue_str}"
        f"Please hand off to SummaryAgent to summarize this movie."
    )



# 4. Summarize movie data
async def summarize_agent(ctx: Context) -> str:
    state = await ctx.get("state")
    details = state.get("movie_details", {})
    budgets = state.get("budget_details", {})
    revenues = state.get("revenue_details", {})

    summary = "🎬 **Movie Summary**\n\n"
    for movie in details.keys():
        summary += f"📌 **{movie}**\n"
        summary += f" - Budget: {budgets.get(movie, 'N/A')}\n"
        summary += f" - Revenue: {revenues.get(movie, 'N/A')}\n"
        summary += f" - Details: {details.get(movie, 'No details')[:300]}...\n\n"

    return summary


search_agent = FunctionAgent(
    name="SearchAgent",
    description="Searches for movie details",
    system_prompt="You find details for a given movie from the movie index.",
    llm=llm,
    tools=[search_movie_details],
    can_handoff_to=["BudgetRevenueAgent","SummaryAgent"]
)

budget_revenue_agent = FunctionAgent(
    name="BudgetRevenueAgent",
    description="Finds both budget and revenue for a given movie.",
    system_prompt=(
        "You are the BudgetRevenueAgent. Your task is to retrieve both the budget and revenue of the given movie "
        "using the provided tool. If movie details are missing, hand off to SearchAgent first. "
        "Once budget and revenue are retrieved, hand off to SummaryAgent."
    ),
    llm=llm,
    tools=[get_movie_budget_and_revenue],
    can_handoff_to=["SearchAgent", "SummaryAgent"]
)


summary_agent = FunctionAgent(
    name="SummaryAgent",
    description="Summarizes the movie with budget, revenue, and key details.",
    system_prompt=(
        "You are the SummaryAgent. Your job is to generate a complete summary of the movie. "
        "Use the stored context to list the budget, revenue, and a brief overview for each movie."
    ),
    llm=llm,
    tools=[summarize_agent],
    can_handoff_to=[]
)

movie_workflow = AgentWorkflow(
    agents=[search_agent, budget_revenue_agent, summary_agent],
    root_agent="SearchAgent",
    initial_state={
        "movie_details": {},
        "budget_details": {},
        "revenue_details": {}
    }
)




In [121]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
query = "tell me about interstellar and itsreview "
query = "tell me about The bat dark knight series and its budget and revenues"
query = "tell me about true lies"
#query = "tell me about a comedy movie"
handler = movie_workflow.run(
    user_msg=(
        query
    )
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # if isinstance(event, AgentStream):
    #     if event.delta:
    #         print(event.delta, end="", flush=True)
    # elif isinstance(event, AgentInput):
    #     print("📥 Input:", event.input)
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: SearchAgent

🛠️  Planning to use tools: ['search_movie_details', 'handoff']
🔨 Calling Tool: search_movie_details
  With arguments: {'movie_name': 'True Lies'}
🔧 Tool Result (search_movie_details):
  Arguments: {'movie_name': 'True Lies'}
  Output: ✅ Movie details found for True Lies.
The movie "True Lies" was released on July 14, 1994. It falls under the genres of Action and Thriller. The film was directed by James Cameron and produced by Twentieth Century Fox Film Corporation and Lightstorm Entertainment. The main cast includes Arnold Schwarzenegger, Jamie Lee Curtis, Tom Arnold, Bill Paxton, and Tia Carrere. The plot revolves around a secret agent who must reveal his identity to his wife and stop nuclear terrorists after they are both kidnapped.
🔨 Calling Tool: handoff
  With arguments: {'to_agent': 'SummaryAgent', 'reason': 'Summarizes the movie with budget, revenue, and key details.'}
🔧 Tool Result (handoff):
  Arguments: {'to_agent': 'SummaryAgent', 'reason': 'Summarizes

In [77]:
response = await movie_workflow.run(query, verbose=True)
print(str(response))


The Dark Knight series, directed by Christopher Nolan, is a trilogy of superhero films based on the DC Comics character Batman. The series includes three films: "Batman Begins" (2005), "The Dark Knight" (2008), and "The Dark Knight Rises" (2012).

1. **Batman Begins (2005)**:
   - **Budget**: Approximately $150 million
   - **Revenue**: Approximately $373 million
   - **Overview**: The film explores the origins of the Batman legend and the emergence of Bruce Wayne as the Dark Knight. After witnessing his parents' murder, Bruce travels the world to understand the nature of crime and becomes Batman to protect Gotham City from the criminal underworld.

2. **The Dark Knight (2008)**:
   - **Budget**: $185 million
   - **Revenue**: $1,004,558,444
   - **Overview**: Batman, with the help of Lt. Jim Gordon and District Attorney Harvey Dent, sets out to dismantle the remaining criminal organizations in Gotham. However, they soon find themselves prey to a reign of chaos unleashed by a rising cr

In [32]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
#query = "Recommend me some sci-fi movies with a rating above 8.0."
query = "Recommend some top movies"
response = await agent_workflow.run(
    user_msg=query
)
print(response)

Here are some top movie recommendations for you:

1. **Top Five** - A comedy-drama about a comedian trying to transition into serious acting.
2. **Top Hat** - A classic musical comedy featuring dance and romance.

Enjoy your movie time!


In [11]:
print(response)

I recommend watching "Sweeney Todd: The Demon Barber of Fleet Street." If you're interested in more TV series recommendations, feel free to ask!
